In [24]:
import pandas as pd
import matplotlib
import numpy as np
import torch

https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow#scrollTo=H5t4_7HdjYEE


1.   pip install transformers 
2.   pip install wandb






In [9]:
df = pd.read_csv('train.csv')
df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [10]:
df_cohesion = df[["text_id", "full_text", "cohesion"]]
df_cohesion

,text_id,full_text,cohesion
0,0016926B079C,I think that students would benefit from learn...,3.5
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0
3,003885A45F42,The best time in life is when you become yours...,4.5
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5
...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0


In [11]:
df_cohesion['rating'] = np.zeros(len(df_cohesion))
for i in range(1, len(df_cohesion)):
    score = df_cohesion.loc[i, 'cohesion']
    if score >= 3.5:
        df_cohesion.loc[i, 'rating'] = 1
    if score <= 2.5:
        df_cohesion.loc[i, 'rating'] = -1

df_cohesion

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,text_id,full_text,cohesion,rating
0,0016926B079C,I think that students would benefit from learn...,3.5,0.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,-1.0
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,0.0
3,003885A45F42,The best time in life is when you become yours...,4.5,1.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,-1.0
...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,-1.0
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,1.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,-1.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,1.0


In [12]:
df_cohesion_sentence = df_cohesion.assign(full_text_sentences=df_cohesion['full_text'].str.split(".").explode('full_text'))
df_cohesion_sentence = df_cohesion_sentence.explode('full_text_sentences')
df_cohesion_sentence = df_cohesion_sentence[df_cohesion_sentence["full_text_sentences"] != '']
df_cohesion_sentence = df_cohesion_sentence.drop(columns = ['full_text'])
df_cohesion_sentence

,text_id,cohesion,rating,full_text_sentences
0,0016926B079C,3.5,0.0,I think that students would benefit from learn...
1,0022683E9EA5,2.5,-1.0,taking only classes helps them because at the...
2,00299B378633,3.0,0.0,they will be comfortable at home
3,003885A45F42,4.5,1.0,\n\nThe hardest part of school is getting ready
4,0049B1DF5CCC,2.5,-1.0,you wake up go brush your teeth and go to you...
...,...,...,...,...
3906,FFD29828A873,2.5,-1.0,\n\nHe asked her how he could make up the poin...
3907,FFD9A83B0849,4.0,1.0,\n\nTwo weeks after that she assigned another ...
3908,FFDC4011AC9C,2.5,-1.0,\n\nShe took Generic_Name aside and said ' thi...
3909,FFE16D704B16,4.0,1.0,\n\nGeneric_Name listed so he asked somebody t...


In [13]:
sentences1 = df_cohesion_sentence["full_text_sentences"].values
labels = df_cohesion_sentence["rating"].values

https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f